In [1]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 11:19:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
'''SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2'''

"SELECT \n    date_trunc('hour', lpep_pickup_datetime) AS hour, \n    PULocationID AS zone,\n\n    SUM(total_amount) AS amount,\n    COUNT(1) AS number_records\nFROM\n    green\nWHERE\n    lpep_pickup_datetime >= '2020-01-01 00:00:00'\nGROUP BY\n    1, 2"

In [3]:
rdd = df_green.select('lpep_pickup_datetime', 'PULocationID', 'total_amount').rdd

In [4]:
from datetime import datetime
from pyspark.sql import types

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [7]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [8]:
from collections import namedtuple

In [9]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'total_amount', 'count'])

In [10]:
def calculate_revenue (left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    output_value = (output_amount, output_count)
    return output_value
    

In [11]:
def unwrap(row):
    return RevenueRow(hour=row[0][0], zone=row [0][1], total_amount=row[1][0], count=row[1][1])

In [12]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('count',types.IntegerType(), True)])

In [13]:
df_result = rdd.filter(filter_outliers).map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue).map(unwrap).toDF(result_schema)

In [14]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('total_amount', DoubleType(), True), StructField('count', IntegerType(), True)])

In [15]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|      total_amount|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [40]:
df_result.write.parquet('tmp/green-revenue')

In [22]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID',  'DOLocationID', 'trip_distance']

duration_rdd = df_green.select(columns) \
    .rdd

In [36]:
import pandas as pd

In [40]:
rows = duration_rdd.take(10)


,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-23 13:10:15,74,130,12.77
1,NaN,2020-01-20 15:09:00,67,39,8.00
2,2.0,2020-01-15 20:23:41,260,157,1.27
3,2.0,2020-01-05 16:32:26,82,83,1.25
4,2.0,2020-01-29 19:22:42,166,42,1.84
5,2.0,2020-01-15 11:07:42,179,223,0.76
6,2.0,2020-01-16 08:22:29,41,237,3.32
7,2.0,2020-01-28 17:05:28,75,161,2.21
8,1.0,2020-01-22 14:51:37,152,166,0.90
9,2.0,2020-01-31 10:25:04,75,234,6.10


In [47]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [52]:
def apply_model_in_batch(rows):
        
    df = pd.DataFrame(rows, columns=columns)
    cnt = len(df)
    prediction = model_predict(df)
    df['predicted_duration'] = prediction
    for row in df.itertuples():
        yield row
           

In [54]:
df_result = duration_rdd.mapPartitions(apply_model_in_batch).toDF().drop('Index')

In [55]:
df_result.show()

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0|                  {}|          74|         130|        12.77|63.849999999999994|
|     NaN|                  {}|          67|          39|          8.0|              40.0|
|     2.0|                  {}|         260|         157|         1.27|              6.35|
|     2.0|                  {}|          82|          83|         1.25|              6.25|
|     2.0|                  {}|         166|          42|         1.84| 9.200000000000001|
|     2.0|                  {}|         179|         223|         0.76|               3.8|
|     2.0|                  {}|          41|         237|         3.32|16.599999999999998|
|     2.0|                  {}|          75|         161|         2.21|             11.05|